In [2]:
import logging
import pandas as pd
import json


In [13]:
def openFile(file_path, mode='r'):
    try:
        return open(file_path, mode)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        raise

def getDataframeFromFile(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('|')
        
        item = {}

        if len(rec) > 6:
            item['movieId'] = rec[0]
            item['name'] = rec[1]
            item['release_date'] = rec[2]
            item['url'] = rec[4]

            #5 => 23
            item['genre'] = [genre[idx]['name'] for  idx, g in enumerate(range(5,24)) if rec[g] == '1']

        elif len(rec) == 2:
            item['id'] = rec[1]
            item['name'] = rec[0]
            
        elif len(rec) == 5:
            item['userId'] = rec[0]
            item['age'] = rec[1]
            item['gender'] = rec[2]
            item['function'] = rec[3]
        else:
            print("Invalid record format. Skipping.")
            return

        data.append(item)
    filed.close()
    return data

def getDataframe(filed):
    data = []
    while True:
        line = filed.readline()
        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('\t')

        item = {}
        item['userId'] = rec[0]
        item['movieId'] = rec[1]
        item['rating'] = rec[2]
        item['timestamp'] = rec[3]
        data.append(item)
    filed.close()
    return data

#=>Data
fileGenre = openFile('../data/u.genre')
fileItem = openFile('../data/u.item')
fileUser = openFile('../data/u.user')
fileData = openFile('../data/u.data')

df_relationship_data = pd.DataFrame(getDataframe(fileData))

genre = getDataframeFromFile(fileGenre)
items = getDataframeFromFile(fileItem)
users = getDataframeFromFile(fileUser)

df_movies = pd.DataFrame(items)
df_users = pd.DataFrame(users)


def tojson(fileName,df):
    data = {}
    data["results"] = df.to_dict(orient='records')
    data["count"] = len(df)
    with open(f'../data/json/{fileName}.json','w') as outputFile:
        json.dump(data,outputFile)


# tojson('movies',df_movies)
# tojson('users',df_users)
# tojson('ratings',df_relationship_data)

#df_relationship_data.to_csv('../data/csv/movie_ratings.csv',index=False)

# df_relationship_data.groupby('userId').apply(lambda x:print(x))